In [29]:
import os
from dotenv import load_dotenv
# Le .env est aussi à la racine
load_dotenv(".env")

USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

import psycopg2
conn = psycopg2.connect(dbname="app_auth", user=USER_PSQL, password=PASSWORD_PSQL, host="localhost", port="5432")

In [30]:
def update_username_from_app(user_id, new_username):
    with conn.cursor() as cur:
        cur.execute("""
            UPDATE "user"
            SET username = %s
            WHERE user_id = %s;
        """, (new_username, user_id))
        conn.commit()
  


In [31]:
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS username_history_from_app (
    username_history_id SERIAL PRIMARY KEY,
    user_id INTEGER REFERENCES "user"(user_id),
    username_new VARCHAR(255),
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
conn.commit()



In [32]:
def update_username_and_history_from_app(user_id, new_username):
    with conn.cursor() as cur:
        # Mise à jour du username
        cur.execute("""
            UPDATE "user"
            SET username = %s
            WHERE user_id = %s
            RETURNING user_id, username;
        """, (new_username, user_id))

        updated_user = cur.fetchone()

        if updated_user:
            cur.execute("""
                INSERT INTO username_history_from_app (user_id, username_new)
                VALUES (%s, %s);
            """, (updated_user[0], updated_user[1]))

        conn.commit()


In [33]:
for num_edits in [100, 1000, 10000]:
    print(f"\nCas avec TRIGGER - {num_edits} éditions:")
    %timeit -n 1 -r 1 [update_username_from_app(random.randint(1, 100), f"new_username_{i}") for i in range(num_edits)]
for num_edits in [100, 1000, 10000]:
    print(f"\nCas avec code Python qui enregistre l'historique - {num_edits} éditions:")
    %timeit -n 1 -r 1 [update_username_and_history_from_app(random.randint(1, 100), f"new_username_{i}") for i in range(num_edits)]
conn.close()


Cas avec TRIGGER - 100 éditions:
119 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

Cas avec TRIGGER - 1000 éditions:
1.11 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

Cas avec TRIGGER - 10000 éditions:
10.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

Cas avec code Python qui enregistre l'historique - 100 éditions:
121 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

Cas avec code Python qui enregistre l'historique - 1000 éditions:
1.14 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

Cas avec code Python qui enregistre l'historique - 10000 éditions:
11.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
